https://towardsdatascience.com/uber-reviews-text-analysis-11613675046

https://www.kaggle.com/purvank/uber-rider-reviews-dataset#Uber_Ride_Reviews.csv

In [14]:
import pandas as pd

import numpy as np

import seaborn as sns

%matplotlib inline

import matplotlib.pyplot as plt

import random

In [8]:
df = pd.read_csv( 'J:\\MyFiles\\Fabrice\\datascience\\data\\Uber_Ride_Reviews.csv' )

In [11]:
df.head( 5 )

,ride_review,ride_rating,sentiment
0,I completed running New York Marathon requeste...,1.0,0
1,My appointment time auto repairs required earl...,1.0,0
2,Whether I using Uber ride service Uber Eats or...,1.0,0
3,Why hard understand I trying retrieve Uber cab...,1.0,0
4,I South Beach FL I staying major hotel ordered...,1.0,0


In [10]:
df.shape

(1344, 3)

In [13]:
df.dtypes

ride_review     object
ride_rating    float64
sentiment        int64
dtype: object

In [19]:
df.describe()

,ride_rating,sentiment
count,1344.000000,1344.000000
mean,1.627232,0.173363
std,1.296149,0.378702
min,1.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,0.000000
max,5.000000,1.000000


In [20]:
df.isnull().sum()

ride_review    0
ride_rating    0
sentiment      0
dtype: int64

In [18]:
df[ 'ride_rating' ].value_counts()

1.0    1041
5.0     124
2.0      70
4.0      59
3.0      50
Name: ride_rating, dtype: int64

In [23]:
# Notations de 1 à 5
# 3 note neutre à effacer

df = df[ df[ 'ride_rating' ] != 3 ]

df[ 'ride_rating' ].value_counts()

1.0    1041
5.0     124
2.0      70
4.0      59
Name: ride_rating, dtype: int64

In [24]:
# Regroupement des notes
# Notes 0 , 1 , 2 > 0 | Notes 4 , 5 > 5

df[ 'binary_ride_rating' ] = np.where( df[ 'ride_rating' ] > 3 , 1 , 0 )


df[ 'binary_ride_rating' ].value_counts()

0    1111
1     183
Name: binary_ride_rating, dtype: int64

### Train/Test Split

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train , X_test , y_train , y_test = train_test_split( df[ 'ride_review' ] , df[ 'binary_ride_rating'] , random_state = 999 )

### Turning words into numbers

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
# w_v : word vector
w_v = CountVectorizer().fit(X_train)

In [34]:
w_v.get_feature_names()[1:10]

['abandoned',
 'abhi',
 'ability',
 'able',
 'about',
 'abrupt',
 'absence',
 'absent',
 'absolute']

In [36]:
len( w_v.get_feature_names() )

6620

In [38]:
#creating matrix array for logistic regression
X_train_v = vect.transform(X_train)

print (X_train_v.toarray() )

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Logistic Regression

In [39]:
from sklearn.linear_model import LogisticRegression

In [43]:
mdl = LogisticRegression( solver = 'liblinear' )

mdl.fit( X_train_v , y_train )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [44]:
from sklearn.metrics import roc_auc_score

In [45]:
y_test_pred = mdl.predict( vect.transform( X_test ) )

In [46]:
roc_auc_score( y_test, y_test_pred )

0.7545248868778279

### Positive and Negative Words

In [48]:
words = np.array(w_v.get_feature_names())

In [50]:
coef_ndx = mdl.coef_[0]

In [56]:
df_word_coef = pd.DataFrame( { 'word' : words , 'coef' : coef_ndx } )

In [57]:
df_word_coef.sort_values( 'coef' ).head( 5 )

,word,coef
924,charged,-1.362451
3830,never,-0.923361
1376,customer,-0.718391
3194,know,-0.677141
1208,contact,-0.613118


In [58]:
df_word_coef.sort_values( 'coef' ).tail( 5 )

,word,coef
6599,you,0.818030
1239,convenient,0.955934
6236,used,0.999332
5023,safe,1.140186
2507,great,1.288327
